# Лабораторная работа №4 (Проведение исследований со случайным лесом)

## Создание бейзлайна и оценка качества

### Обучение моделей из sklearn для выбранных наборов данных

#### Импорт библиотек

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from scipy.spatial import distance
import numpy as np
from collections import Counter

#### Подготовка данных

##### Классификация (Credit Card Fraud Detection Dataset 2023)

**Загрузка данных:**

In [18]:
fraud_data = pd.read_csv('/kaggle/input/credit-card-fraud-detection-dataset-2023/creditcard_2023.csv')
fraud_data = fraud_data.sample(frac=0.05)
X_fraud = fraud_data.iloc[:, :-1]  # Признаки
y_fraud = fraud_data.iloc[:, -1]   # Метки классов

label_encoder = LabelEncoder()
y_fraud_encoded = label_encoder.fit_transform(y_fraud)

**Разделение на тренировочный и тестовый наборы:**

In [19]:
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud_encoded, test_size=0.2, random_state=42)

**Замена пропущенных значений**

In [20]:
def simple_imputer_df(x_train, x_test):
    imputer = SimpleImputer(strategy='mean')
    return pd.DataFrame(imputer.fit_transform(x_train)), pd.DataFrame(imputer.transform(x_test))

In [21]:
X_train_fraud, X_test_fraud = simple_imputer_df(X_train_fraud, X_test_fraud)

**Масштабирование данных**

In [22]:
def scaling(x_train, x_test):
    scaler = StandardScaler()
    return scaler.fit_transform(x_train), scaler.transform(x_test)

In [23]:
X_train_fraud, X_test_fraud = scaling(X_train_fraud, X_test_fraud)

##### Регрессия (Gold Price Regression)

**Загрузка данных:**

In [24]:
gold_data = pd.read_csv('/kaggle/input/financial-data/financial_regression.csv')

X_gold = gold_data.iloc[:, 1:-1]  # Признаки
y_gold = gold_data.iloc[:, -1]   # Целевая переменная

**Разделение на тренировочный и тестовый наборы:**

In [25]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)

**Замена пропущенных значений:**

In [26]:
def simple_imputer(x_train, x_test):
    imputer = SimpleImputer(strategy='mean')
    return imputer.fit_transform(x_train), imputer.transform(x_test)

In [27]:
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)

**Удаление строк с пропущенными значениями:**

In [28]:
def delete_none(x_train, y_train, x_test, y_test):
    mask = ~np.isnan(y_train)
    mask_test = ~np.isnan(y_test)
    return x_train[mask], y_train[mask], x_test[mask_test], y_test[mask_test]

In [29]:
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

#### Обучение моделей

##### Классификация

In [30]:
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train_fraud, y_train_fraud)

RandomForestClassifier(random_state=42)

##### Регрессия

In [31]:
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train_gold, y_train_gold)

RandomForestRegressor(random_state=42)

### Оценка качества моделей по выбранным метрикам

##### Классификация

In [32]:
def quality_eval_classification(X_test, y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    return accuracy, f1

In [33]:
accuracy_origin, f1_origin = quality_eval_classification(X_test_fraud, y_test_fraud, classifier)

Accuracy: 0.9994724810972393
F1 Score: 0.9994724826308604


##### Регрессия

In [34]:
def quality_eval_regression(X_test, y_test, model):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print(f"R-squared: {r2}")
    return mae, r2

In [35]:
mae_origin, r2_origin = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 2444766.2500400534
R-squared: 0.610479164922296


## Улучшение бейзлайна

### Гипотезы

1. Подбор гиперпараметров:
   - Использование Grid Search с кросс-валидацией для оптимизации модели.

### Проверка гипотез

#### 1. Подбор гиперпараметров

##### Классификация

Подготовка данных:

In [36]:
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud_encoded, test_size=0.2, random_state=42)
X_train_fraud, X_test_fraud = simple_imputer_df(X_train_fraud, X_test_fraud)
X_train_fraud, X_test_fraud = scaling(X_train_fraud, X_test_fraud)

Определение параметров для перебора:

In [37]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

Настройка и обучение GridSearchCV:

In [38]:
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_fraud, y_train_fraud)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]})

Вывод лучших параметров и результата:

In [39]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Best cross-validation accuracy:  0.9997361941600774


Оценка качества модели:

In [40]:
quality_eval_classification(X_test_fraud, y_test_fraud, grid_search.best_estimator_)

Accuracy: 0.9994724810972393
F1 Score: 0.9994724826308604


(0.9994724810972393, 0.9994724826308604)

##### Регрессия

Подготовка данных:

In [41]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

Определение параметров для перебора:

In [42]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

Настройка и обучение GridSearchCV:

In [43]:
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_gold, y_train_gold)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]})

Вывод лучших параметров и результата:

In [44]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Best cross-validation accuracy:  0.6640723937986626


Оценка модели с лучшими параметрами на тестовом наборе:

In [45]:
quality_eval_regression(X_test_gold, y_test_gold, grid_search.best_estimator_)

Mean Absolute Error: 2392735.5308633246
R-squared: 0.6161405634184933


(2392735.5308633246, 0.6161405634184933)

### Формирование улучшенного бейзлайна

##### Классификация

Формирование улучшенного бейзлайна:

In [46]:
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud_encoded, test_size=0.2, random_state=42)
X_train_fraud, X_test_fraud = simple_imputer_df(X_train_fraud, X_test_fraud)
X_train_fraud, X_test_fraud = scaling(X_train_fraud, X_test_fraud)
                                   
classifier = RandomForestClassifier(random_state=42, max_depth=None, min_samples_split=2, n_estimators=150)

##### Регрессия

In [47]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

regressor = RandomForestRegressor(random_state=42, max_depth=20, max_features="sqrt", min_samples_leaf=1, min_samples_split=2, n_estimators=150)

### Обучение модели с улучшенным бейзлайном

##### Классификация

Обучение модели:

In [48]:
classifier.fit(X_train_fraud, y_train_fraud)

RandomForestClassifier(n_estimators=150, random_state=42)

##### Регрессия

Обучение модели:

In [49]:
regressor.fit(X_train_gold, y_train_gold)

RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=150,
                      random_state=42)

### Оценка качества модели с улучшенным бейзлайном

##### Классификация

Оценка качества модели:

In [50]:
accuracy_improved, f1_improved = quality_eval_classification(X_test_fraud, y_test_fraud, classifier)

Accuracy: 0.9994724810972393
F1 Score: 0.9994724826308604


##### Регрессия

Оценка качества модели:

In [51]:
mae_improved, r2_improved = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 2392735.5308633246
R-squared: 0.6161405634184933


### Сравнение результатов первоначального бейзлайна с улучшенным бейзлайном

##### Классификация

Видим, что результаты почти идентичны:

In [52]:
print(f"Accuracy difference: {accuracy_improved-accuracy_origin}")
print(f"F1 Score difference: {f1_improved-f1_origin}")

Accuracy difference: 0.0
F1 Score difference: 0.0


##### Регрессия

Видим, что результаты немного улучшились:

In [53]:
print(f"Mean Absolute Error difference: {mae_origin-mae_improved}")
print(f"R-squared difference: {r2_improved-r2_origin}")

Mean Absolute Error difference: 52030.719176728744
R-squared difference: 0.00566139849619729


### Выводы

Улучшенный бейзлайн оказался лучше базового, хотя и не так сильно. Скорей всего, нужно выдвигать и проверять другие гипотезы.

## Имплементация алгоритма машинного обучения

### Самостоятельная имплементация

Случайный лес (Random Forest) — это ансамблевый алгоритм машинного обучения, использующий метод объединения множества базовых моделей (в данном случае деревьев решений) для улучшения общей производительности. 

##### Классификация

Сначала реализауем дерево решений

In [54]:
class DecisionTreeClassifierCustom:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def _gini(self, y):
        m = len(y)
        return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))

    def _best_split(self, X, y):
        # Placeholder for storing best parameters
        best_idx, best_thr = None, None
        best_gini = 1.0
        
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = [np.sum(y == c) for c in range(self.n_classes_)]
            
            for i in range(1, len(y)):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                
                gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in range(self.n_classes_))
                gini_right = 1.0 - sum((num_right[x] / (len(y) - i)) ** 2 for x in range(self.n_classes_))
                
                gini = (i * gini_left + (len(y) - i) * gini_right) / len(y)
                
                if thresholds[i] == thresholds[i - 1]:
                    continue
                
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = NodeClassifierCustom(
            gini=self._gini(y),
            num_samples=len(y),
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class
        )

        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
        
class NodeClassifierCustom:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0 
        self.left = None
        self.right = None

Теперь используем наше дерево решений для создания ансамбля — случайного леса:

In [55]:
class RandomForestClassifierCustom:
    def __init__(self, n_trees=10, max_depth=None, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.n_features = n_features
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTreeClassifierCustom(max_depth=self.max_depth)
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        return [Counter(tree_pred).most_common(1)[0][0] for tree_pred in tree_preds]

##### Регрессия

Сначала реализауем дерево решений

In [56]:
class DecisionTreeRegressorCustom:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def _mse(self, y):
        if len(y) == 0:
            return 0
        return np.mean((y - np.mean(y)) ** 2)

    def _best_split(self, X, y):
        best_idx, best_thr = None, None
        best_mse = np.inf

        for idx in range(self.n_features_):
            thresholds, values = zip(*sorted(zip(X[:, idx], y)))
            mse_left = mse_right = 0
            sum_left = sum_right = 0
            n_left = n_right = 0

            for i in range(1, len(y)):
                sum_left += values[i - 1]
                mse_left = np.mean((values[:i] - np.mean(values[:i])) ** 2) if i > 0 else 0

                sum_right -= values[i - 1]
                mse_right = np.mean((values[i:] - np.mean(values[i:])) ** 2) if i < len(y) else 0
                
                mse = (i * mse_left + (len(y) - i) * mse_right) / len(y)

                if thresholds[i] == thresholds[i - 1]:
                    continue

                if mse < best_mse:
                    best_mse = mse
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        node_value = np.mean(y)
        node = NodeRegressorCustom(
            num_samples=len(y),
            node_value=node_value
        )

        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.node_value

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

class NodeRegressorCustom:
    def __init__(self, num_samples, node_value):
        self.num_samples = num_samples
        self.node_value = node_value
        self.feature_index = 0
        self.threshold = 0 
        self.left = None
        self.right = None

Теперь используем наше дерево решений для создания ансамбля — случайного леса:

In [57]:
class RandomForestRegressorCustom:
    def __init__(self, n_trees=10, max_depth=None, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.n_features = n_features
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTreeRegressorCustom(max_depth=self.max_depth)
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_preds, axis=0)

### Обучение имплементированной модели

##### Классификация

Подготовка данных:

In [58]:
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud_encoded, test_size=0.2, random_state=42)
X_train_fraud, X_test_fraud = simple_imputer_df(X_train_fraud, X_test_fraud)
X_train_fraud, X_test_fraud = scaling(X_train_fraud, X_test_fraud)

classifier = RandomForestClassifierCustom(max_depth=5)

Обучение модели:

In [59]:
classifier.fit(X_train_fraud, y_train_fraud)

##### Регрессия

Подготовка данных:

In [60]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
y_train_gold = y_train_gold.to_numpy()
y_test_gold = y_test_gold.to_numpy()
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

regressor = RandomForestRegressorCustom(max_depth=5)

Обучение модели:

In [61]:
regressor.fit(X_train_gold, y_train_gold)

### Оценка качества имплементированных моделей

##### Классификация

Оценка качества модели:

In [62]:
accuracy_my, f1_my = quality_eval_classification(X_test_fraud, y_test_fraud, classifier)

Accuracy: 0.9994724810972393
F1 Score: 0.9994724826308604


##### Регрессия

Оценка качества модели:

In [63]:
mae_my, r2_my = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 2804569.3010066105
R-squared: 0.548955021519389


### Сравнение результатов первоначального бейзлайна и имплементированных моделей

##### Классификация

Видим, что результаты имплементированной модели немного хуже:

In [64]:
print(f"Accuracy difference: {accuracy_my-accuracy_origin}")
print(f"F1 Score difference: {f1_my-f1_origin}")

Accuracy difference: 0.0
F1 Score difference: 0.0


##### Регрессия

Видим, что результаты имплементированной модели хуже:

In [65]:
print(f"Mean Absolute Error difference: {mae_my-mae_origin}")
print(f"R-squared difference: {r2_my-r2_origin}")

Mean Absolute Error difference: 359803.0509665571
R-squared difference: -0.06152414340290702


### Выводы

Оценка качества наших имплементированных моделей оказалась хуже.
Можно сделать следующие выводы:
1. Предобработка данных важна: Масштабирование и нормализация данных могут значительно повлиять на качество модели.
2. Оптимизация гиперпараметров: Правильная настройка параметров, таких как темп обучения и количество итераций, необходима для достижения оптимальной производительности.
3. Регуляризация: Помогает избежать переобучения и может улучшить обобщающую способность модели.
4. Алгоритмическая точность: Правильная реализация алгоритма и внимание к численным ошибкам критичны для качества модели.
5. Используйте проверенные решения: Изучение и сравнение с библиотечными реализациями может помочь выявить слабые места вашей модели.
6. Тщательно изучайте данные: Соблюдение чистоты, отсутствие пропусков и ошибок в данных — важный аспект построения качественной модели.

### Добавление техник из улучшенного бейзлайна

##### Классификация

In [66]:
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud_encoded, test_size=0.2, random_state=42)
X_train_fraud, X_test_fraud = simple_imputer_df(X_train_fraud, X_test_fraud)
X_train_fraud, X_test_fraud = scaling(X_train_fraud, X_test_fraud)

classifier = RandomForestClassifierCustom(max_depth=5, n_trees=20)

##### Регрессия

Формирование улучшенного бейзлайна:

In [67]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
y_train_gold = y_train_gold.to_numpy()
y_test_gold = y_test_gold.to_numpy()
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

regressor = RandomForestRegressorCustom(max_depth=20, n_trees=10)

### Обучение на улучшенном бейзлайне

##### Классификация

In [68]:
classifier.fit(X_train_fraud, y_train_fraud)

##### Регрессия

In [69]:
regressor.fit(X_train_gold, y_train_gold)

### Оценка качества моделей на улучшенном бейзлайне

##### Классификация

In [70]:
accuracy_my_improved, f1_my_improved = quality_eval_classification(X_test_fraud, y_test_fraud, classifier)

Accuracy: 0.9992966414629858
F1 Score: 0.9992966414629858


##### Регрессия

In [71]:
mae_my_impoved, r2_my_impoved = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 2596286.5584117044
R-squared: 0.5407459833651438


### Сравнение результатов

##### Классификация

In [72]:
 print(f"Accuracy difference: {accuracy_my_improved-accuracy_improved}")
print(f"F1 Score difference: {f1_my_improved-f1_improved}")

Accuracy difference: -0.00017583963425349403
F1 Score difference: -0.00017584116787461301


##### Регрессия

In [73]:
print(f"Mean Absolute Error difference: {mae_my_impoved-mae_improved}")
print(f"R-squared difference: {r2_my_impoved-r2_improved}")

Mean Absolute Error difference: 203551.02754837973
R-squared difference: -0.07539458005334954


### Выводы

Улучшенный бейзлайн собственной имплементации получился немного хуже улучшенного бейзлайна готовой реализации. Вероятно, это из-за того, что было выбрано меньше итераций для обучения модели собственной имплементации.